文件转换为csv格式

In [1]:
import os
import pandas as pd

# 获取文件夹中的所有文件
folder_path = r'C:\Users\20676\Desktop\大伙房场次洪水\大伙房水库'
files = os.listdir(folder_path)

# 对于每个文件
for file in files:
    # 检查是否为xlsx文件
    if file.endswith('.xlsx'):
        # 读取Excel文件
        df = pd.read_excel(os.path.join(folder_path, file))
        # 将数据保存为CSV文件，文件名与原文件相同，但扩展名为.csv
        df.to_csv(os.path.join(folder_path, file[:-5] + '.csv'), index=False)
        # 删除原来的Excel文件
        os.remove(os.path.join(folder_path, file))

只保留实测入流列和预报总入流列（预报总入流列将会在绘图的时候用到）

In [2]:
import os
import pandas as pd

# 获取文件夹中的所有文件
folder_path = r'C:\Users\20676\Desktop\大伙房场次洪水\大伙房水库'
files = os.listdir(folder_path)

# 对于每个文件
for file in files:
    # 检查是否为csv文件
    if file.endswith('.csv'):
        # 读取CSV文件
        df = pd.read_csv(os.path.join(folder_path, file))
        # 提取文件名中的年份
        year = file[-12:-8]
        # 添加年份到时间列
        df['时间'] = year + '-' + df['时间']
        # 只保留时间列和实测入流列
        df = df[['时间', '实测入流','预报总入流']]
        # 保存回CSV文件
        df.to_csv(os.path.join(folder_path, file), index=False)

只保留实测入流列

In [ ]:
import os
import pandas as pd

# 指定的文件夹路径
folder_path = r'C:\Users\20676\Desktop\大伙房场次洪水\大伙房水库'
save_folder_path = r'C:\Users\20676\Desktop\大伙房场次洪水\测试'

# 列出文件夹中的所有文件
files = os.listdir(folder_path)

# 遍历所有文件
for file in files:
    # 检查文件是否为csv文件
    if file.endswith('.csv'):
        # 读取csv文件
        df = pd.read_csv(os.path.join(folder_path, file))
        # 选择时间列和实测入流列
        df = df[['时间', '实测入流']]
        # 保存新的DataFrame到指定的文件夹中
        df.to_csv(os.path.join(save_folder_path, file), index=False)

输出重采样并且线性插值

In [ ]:
import os
import pandas as pd

# 获取文件夹中的所有文件
folder_path = r'C:\Users\20676\Desktop\大伙房场次洪水\测试'
files = os.listdir(folder_path)

# 对于每个文件
for file in files:
    # 检查是否为csv文件
    if file.endswith('.csv'):
        # 读取CSV文件
        df = pd.read_csv(os.path.join(folder_path, file))
        # 将时间列转换为Datetime类型，并设置为索引
        df['时间'] = pd.to_datetime(df['时间'])
        df = df.set_index('时间')
        # 重采样时间列为每小时的，对于缺失的时间点，实测入流列的值将为NaN
        df = df.resample('H').asfreq()
        # 对实测入流列进行线性插值
        df['实测入流'] = df['实测入流'].interpolate(method='linear')
        # 将实测入流列的值四舍五入到一位小数
        df['实测入流'] = df['实测入流'].round(1)
        # 将时间列从索引转换回普通列
        df = df.reset_index()
        # 保存回CSV文件
        df.to_csv(os.path.join(folder_path, file), index=False)

In [ ]:
import os
import pandas as pd

# 指定的文件夹路径
folder_path = r'C:\Users\20676\Desktop\大伙房场次洪水\测试'

# 列出文件夹中的所有文件
files = os.listdir(folder_path)

# 遍历所有文件
for file in files:
    # 检查文件是否为csv文件
    if file.endswith('.csv'):
        # 读取csv文件
        df = pd.read_csv(os.path.join(folder_path, file))
        # 删除实测入流列为空的行
        df = df.dropna(subset=['实测入流'])
        # 将清理后的DataFrame保存回csv文件
        df.to_csv(os.path.join(folder_path, file), index=False)

创建新的时间范围，场次洪水及其前一个月

In [ ]:
import os
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta

# 指定的文件夹路径
folder_path = r'C:\Users\20676\Desktop\大伙房场次洪水\测试'

# 列出文件夹中的所有文件
files = os.listdir(folder_path)

# 遍历所有文件
for file in files:
    # 检查文件是否为csv文件
    if file.endswith('.csv'):
        # 读取csv文件
        df = pd.read_csv(os.path.join(folder_path, file))
        # 将时间列转换为datetime对象
        df['时间'] = pd.to_datetime(df['时间'])
        # 获取时间列的第一个时间
        first_time = df['时间'].iloc[0]
        # 创建一个新的时间范围，从一个月前开始，到原始时间结束，间隔为一小时
        new_time_range = pd.date_range(start=first_time - relativedelta(months=1), end=first_time, freq='h')
        # 创建一个新的DataFrame，使用新的时间范围作为时间列，实测入流列为NaN
        new_df = pd.DataFrame({'时间': new_time_range})
        new_df['实测入流'] = np.nan
        # 将原始DataFrame附加到新的DataFrame上
        new_df = pd.concat([new_df, df], ignore_index=True)
        # 将新的DataFrame保存回csv文件
        new_df.to_csv(os.path.join(folder_path, file), index=False)

拿到按年处理的数据里面得到的逐小时雨量表格

In [ ]:
import pandas as pd

# 读取逐小时雨量.csv文件
df = pd.read_csv(r'C:\Users\20676\Desktop\大伙房场次洪水\逐小时雨量.csv')

# 修改列名
df = df.rename(columns={'时间': 'time', '面雨量': 'prcp(mm/hour)'})

# 保存回雨量.csv文件
df.to_csv(r'C:\Users\20676\Desktop\大伙房场次洪水\逐小时雨量.csv', index=False)

将逐小时雨量填入

In [ ]:
import os
import pandas as pd

# 指定的文件夹路径
folder_path = r'C:\Users\20676\Desktop\大伙房场次洪水\测试'
rainfall_file_path = r'C:\Users\20676\Desktop\大伙房场次洪水\逐小时雨量.csv'

# 读取逐小时雨量.csv文件
rainfall_df = pd.read_csv(rainfall_file_path)
rainfall_df['time'] = pd.to_datetime(rainfall_df['time'])

# 列出文件夹中的所有文件
files = os.listdir(folder_path)

# 遍历所有文件
for file in files:
    # 检查文件是否为csv文件
    if file.endswith('.csv'):
        # 读取csv文件
        df = pd.read_csv(os.path.join(folder_path, file))
        df['时间'] = pd.to_datetime(df['时间'])
        # 在逐小时雨量.csv文件中找到时间列对应的时间，提取对应的prcp(mm/hour)列所在的行的数据
        df = df.merge(rainfall_df, left_on='时间', right_on='time', how='left')
        # 删除多余的time列
        df = df.drop(columns=['time'])
        # 将更新后的DataFrame保存回csv文件
        df.to_csv(os.path.join(folder_path, file), index=False)

上面填入的时候出问题，多余重复的行，所以删除重复的时间行

In [ ]:
import os
import pandas as pd

# 指定的文件夹路径
folder_path = r'C:\Users\20676\Desktop\大伙房场次洪水\测试'

# 列出文件夹中的所有文件
files = os.listdir(folder_path)

# 遍历所有文件
for file in files:
    # 检查文件是否为csv文件
    if file.endswith('.csv'):
        # 文件路径
        file_path = os.path.join(folder_path, file)
        # 读取csv文件
        df = pd.read_csv(file_path)
        # 删除时间列重复的行
        df = df.drop_duplicates(subset='时间', keep='last')
        # 将处理后的DataFrame保存回原csv文件
        df.to_csv(file_path, index=False)

改名

In [ ]:
import os
import pandas as pd

# 指定的文件夹路径
folder_path = r'C:\Users\20676\Desktop\大伙房场次洪水\测试'

# 列出文件夹中的所有文件
files = os.listdir(folder_path)

# 遍历所有文件
for file in files:
    # 检查文件是否为csv文件
    if file.endswith('.csv'):
        # 读取csv文件
        df = pd.read_csv(os.path.join(folder_path, file))
        # 将'时间'列改为'time'，将'实测入流'列改为'flow(m^3/s)'
        df = df.rename(columns={'时间': 'time', '实测入流': 'flow(m^3/s)'})
        # 将修改后的DataFrame保存回csv文件
        df.to_csv(os.path.join(folder_path, file), index=False)

填充潜在蒸发

In [ ]:
import os
import pandas as pd

# 指定的文件夹路径
folder_path = r'C:\Users\20676\Desktop\大伙房场次洪水\测试'

# 根据时间列的月份创建新的列
def calculate_pet(row):
    month = row['time'].month
    if month == 5:
        return 99.3 / 31 / 24
    elif month == 6:
        return 102.2 / 30 / 24
    elif month == 7:
        return 100.3 / 31 / 24
    elif month == 8:
        return  92.2/ 31 / 24
    elif month == 9:
        return 72.4 / 30 / 24
    else:
        return 0

# 列出文件夹中的所有文件
files = os.listdir(folder_path)

# 遍历所有文件
for file in files:
    # 检查文件是否为csv文件
    if file.endswith('.csv'):
        # 读取csv文件
        df = pd.read_csv(os.path.join(folder_path, file))
        # 将时间列转换为日期时间格式
        df['time'] = pd.to_datetime(df['time'])
        # 计算'pet(mm/hour)'列的值
        df['pet(mm/hour)'] = df.apply(calculate_pet, axis=1)
        # 将新列的值四舍五入到两位小数
        df['pet(mm/hour)'] = df['pet(mm/hour)'].round(3)
        # 将修改后的DataFrame保存回csv文件
        df.to_csv(os.path.join(folder_path, file), index=False)

将场次洪水拼接

In [ ]:
import os
import pandas as pd

# 指定的文件夹路径
folder_path = r'C:\Users\20676\Desktop\大伙房场次洪水\测试'

# 列出文件夹中的所有文件
files = os.listdir(folder_path)

# 创建一个空的DataFrame列表
dfs = []

# 遍历所有文件
for file in files:
    # 检查文件是否为csv文件
    if file.endswith('.csv'):
        # 读取csv文件
        df = pd.read_csv(os.path.join(folder_path, file))
        # 将DataFrame添加到列表中
        dfs.append(df)

# 使用concat方法将所有的DataFrame拼接起来
df_all = pd.concat(dfs)

# 将拼接后的DataFrame保存为basin_21100150.csv文件
df_all.to_csv(os.path.join(folder_path, 'basin_21100150.csv'), index=False)

重设时间

In [ ]:
import pandas as pd
import numpy as np

# 文件路径
file_path = r'C:\Users\20676\Desktop\大伙房场次洪水\测试\basin_21100150.csv'

# 读取csv文件
df = pd.read_csv(file_path)

# 创建一个新的时间序列，从2010-01-01 00:00:00开始，逐小时增加，长度与原数据相同
new_time = pd.date_range(start='2010-01-01 00:00:00', periods=len(df), freq='H')

# 将新的时间序列设置为数据的时间列
df['time'] = new_time

# 将修改后的数据保存回csv文件
df.to_csv(file_path, index=False)